# Covid-19 Suivi temps réel GLOBAL

###### Code - Import librairies

In [140]:
import glob, os    
from datetime import date

import warnings
warnings.filterwarnings('ignore')

# Import lecture/stockage données
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import calmap
import folium
from IPython.core.display import display, HTML


In [66]:
# Palette
colorGrey = '#393e46'
colorRed = '#ff2e63' 
colorCyan = '#21bf73' 
colorYellow = '#fe9801'

###### Code - Chargement des données

In [11]:
today = date.today()
d1 = today.strftime("%m-%d-%Y")
print("d1 =", str(d1))

d1 = 03-19-2020


In [24]:
# On charge et map l'ensemble des csv en un seul
# dfGlobalDaily = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv"))))

# dfGlobalDailyLatest = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed/" + str(d1) + ".csv")
dfGlobalDailyLatest = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-18-2020.csv")

# Chargement des times series pour les cas confirmés
dfGlobalSeriesConfirmed = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")

# Chargement des times series pour les cas décédés
dfGlobalSeriesDeaths = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")

# Chargement des times series pour les cas soignés
dfGlobalSeriesRecovered = pd.read_csv("../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

## Cas Confirmé/Morts/Soignés par pays

###### Code

In [13]:
# Cas ACTIVE = cas confirmé - cas décédés - cas soignés
dfGlobalDailyLatest['Active'] = dfGlobalDailyLatest['Confirmed'] - dfGlobalDailyLatest['Deaths'] - dfGlobalDailyLatest['Recovered']
globalAll = dfGlobalDailyLatest.groupby(['Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum()


In [14]:
classement = globalAll.sort_values(by='Confirmed', ascending=False)
classement = classement.reset_index(drop=False)
classement.style.background_gradient(cmap='Reds')

,Country/Region,Confirmed,Deaths,Recovered,Active
0,China,81102,3241,69755,8106
1,Italy,35713,2978,4025,28710
2,Iran,17361,1135,5389,10837
3,Spain,13910,623,1081,12206
4,Germany,12327,28,105,12194
5,France,9052,148,12,8892
6,"Korea, South",8413,84,1540,6789
7,US,7786,118,106,7562
8,Switzerland,3028,28,15,2985
9,United Kingdom,2642,72,67,2503


## Nombre de pays contaminé au virus au cours du temps


###### Code

## Top 10 pays


In [142]:
display(HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1619129"><script src="https://public.flourish.studio/resources/embed.js"></script></div>'))

### Décédés

In [81]:
top10Deaths = px.bar(classement.sort_values('Deaths', ascending=False).head(10).sort_values('Deaths', ascending=True), 
             x="Deaths", y="Country/Region", text='Deaths', orientation='h', 
             width=800, height=500, range_x = [0, max(classement['Deaths'])+500])
top10Deaths.update_traces(marker_color=colorRed, opacity=0.6, textposition='outside')
top10Deaths.show()

### Actifs

In [82]:
top10Active = px.bar(classement.sort_values('Active', ascending=False).head(10).sort_values('Active', ascending=True), 
             x="Active", y="Country/Region", text='Active', orientation='h', 
             width=800, height=500, range_x = [0, max(classement['Active'])+5000])
top10Active.update_traces(marker_color=colorGrey, opacity=0.6, textposition='outside')
top10Active.show()

### Confirmés

In [68]:
top10Confirmed = px.bar(classement.sort_values('Confirmed', ascending=False).head(10).sort_values('Confirmed', ascending=True), 
             x="Confirmed", y="Country/Region", text='Confirmed', orientation='h', 
             width=800, height=500, range_x = [0, max(classement['Confirmed'])+8000])
top10Confirmed.update_traces(marker_color=colorYellow, opacity=0.6, textposition='outside')
top10Confirmed.show()


### Soignés

In [86]:
top10Recovered = px.bar(classement.sort_values('Recovered', ascending=False).head(10).sort_values('Recovered', ascending=True), 
             x="Recovered", y="Country/Region", text='Recovered', orientation='h', 
             width=800, height=500, range_x = [0, max(classement['Recovered'])+10000])
top10Recovered.update_traces(marker_color=colorCyan, opacity=0.6, textposition='outside')
top10Recovered.show()

### Taux mortalité pour 100 cas confirmés

In [88]:
top10MortalityRate = classement

# Ajout de la nouvelle colonne tauxMortalité = (Morts/Confirmé)*100, arrondi à deux decimals
top10MortalityRate['Mortality Rate'] = round((top10MortalityRate['Deaths']/top10MortalityRate['Confirmed'])*100, 2)

# On prend que les pays qui ont plus de 100 cas confirmés
temp = top10MortalityRate[top10MortalityRate['Confirmed']>100]
temp = temp.sort_values('Mortality Rate', ascending=False)

fig = px.bar(temp.sort_values('Mortality Rate', ascending=False).head(15).sort_values('Mortality Rate', ascending=True), 
             x="Mortality Rate", y="Country/Region", text='Mortality Rate', orientation='h', 
             width=800, height=500, range_x = [0, 10])
fig.update_traces(marker_color=colorRed, opacity=0.6, textposition='outside')
fig.show()

source data : https://github.com/CSSEGISandData/COVID-19